In [1051]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.utils import plot_model #모델 시각화 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


tf.config.run_functions_eagerly(False)



## 데이터 불러오기
- 앞서 크롤링하여 엑셀에 저장한 파일을 불러온다.

In [1052]:
data_df = pd.read_excel('baseball_data.xlsx')
target_df = pd.read_excel('label.xlsx')
try:
    del data_df['Unnamed: 0']
    del target_df['Unnamed: 0']
except:
    pass


## 훈련세트, 검증세트, 테스트세트 분리
- 8:2비율로 훈련세트, 테스트 세트를 분리하고 또 다시 8:2로 훈련세트, 검증세트를 분리한다.

In [1053]:
#훈련세트, 검증세트, 테스 트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)
x_train, x_val, y_train, y_val = train_test_split(X_full_set,y_full_set, test_size=0.2, random_state=7)
display(x_train)

,타석,타수,안타,홈런,볼넷,사구,삼진,땅볼,플라이,팀타율,잔루,상대선발투수ERA,상대선발투수WHIP
1362,42,34,10,2,6,1,9,9,6,0.294118,9,9.55,2.19
3844,37,33,7,0,2,2,12,8,6,0.212121,12,4.56,1.51
3695,37,34,7,1,3,0,10,9,8,0.205882,19,2.45,1.17
3424,37,35,10,1,1,1,10,9,6,0.285714,15,6.57,1.91
2934,46,38,11,0,7,1,7,12,8,0.289474,30,4.82,1.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3532,37,36,11,1,0,1,6,6,13,0.305556,11,2.72,1.11
84,39,36,10,0,2,0,7,9,10,0.277778,15,1.83,1.00
1600,48,44,19,1,4,0,12,7,6,0.431818,22,8.10,2.10
1287,36,34,9,2,2,0,8,5,12,0.264706,13,4.35,1.43


## 데이터 전처리
- min-max 스케일링(정규화) 와 표준화 둘 중 더 좋은걸로 선택
- 전처리 하지 않는걸로...

In [1054]:
#데이터 정규화



scaler = StandardScaler().fit(X_full_set)

X_full_set = scaler.transform(X_full_set)

x_train = scaler.transform(x_train)
x_val=scaler.transform(x_val)
X_test = scaler.transform(X_test)



# 하이퍼파라미터 튜닝

In [1061]:
def build_model(n_hidden=4, n_neurons=400, learning_rate=1e-3, input_shape=[13]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(x_train, y_train, epochs=100,
                  validation_data=(x_val, y_val),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100


C:\Users\admin\AppData\Local\Temp\ipykernel_14216\2324621434.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


52/52 [==============================] - 0s 3ms/step - loss: 32.7408 - val_loss: 25.1914
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 21.3752 - val_loss: 17.5499
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 15.4605 - val_loss: 13.1855
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 11.7778 - val_loss: 10.2959
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 9.2350 - val_loss: 8.2944
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 7.4474 - val_loss: 6.8794
Epoch 7/100
52/52 [==============================] - 0s 2ms/step - loss: 6.1705 - val_loss: 5.8605
Epoch 8/100
52/52 [==============================] - 0s 2ms/step - loss: 5.2531 - val_loss: 5.1059
Epoch 9/100
52/52 [==============================] - 0s 2ms/step - loss: 4.5723 - val_loss: 4.5520
Epoch 10/100
52/52 [==============================] - 0s 2ms/step - loss: 4.0747 - val_loss: 4.1490
Epoch 11/100


52/52 [==============================] - 0s 2ms/step - loss: 2.4540 - val_loss: 2.6493
Epoch 84/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4543 - val_loss: 2.6493
Epoch 85/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4531 - val_loss: 2.6489
Epoch 86/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4524 - val_loss: 2.6486
Epoch 87/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4527 - val_loss: 2.6467
Epoch 88/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4518 - val_loss: 2.6500
Epoch 89/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4526 - val_loss: 2.6482
Epoch 90/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4517 - val_loss: 2.6464
Epoch 91/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4512 - val_loss: 2.6451
Epoch 92/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4499 - val_loss: 2.6441
Epoch 93/100


52/52 [==============================] - 0s 2ms/step - loss: 2.4832 - val_loss: 2.6639
Epoch 64/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4823 - val_loss: 2.6647
Epoch 65/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4825 - val_loss: 2.6643
Epoch 66/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4819 - val_loss: 2.6634
Epoch 67/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4815 - val_loss: 2.6624
Epoch 68/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4808 - val_loss: 2.6645
Epoch 69/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4805 - val_loss: 2.6617
Epoch 70/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4808 - val_loss: 2.6599
Epoch 71/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4797 - val_loss: 2.6579
Epoch 72/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4795 - val_loss: 2.6569
Epoch 73/100


...

52/52 [==============================] - 0s 2ms/step - loss: 1.9222 - val_loss: 2.4619
Epoch 29/100
52/52 [==============================] - 0s 2ms/step - loss: 1.9054 - val_loss: 2.4642
Epoch 30/100
26/26 [==============================] - 0s 999us/step - loss: 2.4997
[CV] END learning_rate=0.006383466440191768, n_hidden=3, n_neurons=96; total time=   4.0s
Epoch 1/100
52/52 [==============================] - 0s 3ms/step - loss: 13.4922 - val_loss: 4.2150
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 4.0320 - val_loss: 3.4527
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 3.3558 - val_loss: 3.1335
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 3.0413 - val_loss: 2.9171
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 2.8340 - val_loss: 2.7813
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 2.7250 - val_loss: 2.7167
Epoch 7/100
52/52 [==========================

Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 3.6279 - val_loss: 3.1920
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 3.2171 - val_loss: 2.9963
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 2.9624 - val_loss: 2.8553
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 2.7990 - val_loss: 2.7415
Epoch 7/100
52/52 [==============================] - 0s 2ms/step - loss: 2.6858 - val_loss: 2.6855
Epoch 8/100
52/52 [==============================] - 0s 2ms/step - loss: 2.6134 - val_loss: 2.6504
Epoch 9/100
52/52 [==============================] - 0s 2ms/step - loss: 2.5575 - val_loss: 2.5937
Epoch 10/100
52/52 [==============================] - 0s 2ms/step - loss: 2.5121 - val_loss: 2.5744
Epoch 11/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4799 - val_loss: 2.5424
Epoch 12/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4444 - val_loss: 2.5313
Epoch 1

52/52 [==============================] - 0s 2ms/step - loss: 2.5086 - val_loss: 2.6920
Epoch 10/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4661 - val_loss: 2.6655
Epoch 11/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4302 - val_loss: 2.6366
Epoch 12/100
52/52 [==============================] - 0s 2ms/step - loss: 2.4028 - val_loss: 2.6338
Epoch 13/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3800 - val_loss: 2.6136
Epoch 14/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3660 - val_loss: 2.5912
Epoch 15/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3496 - val_loss: 2.5860
Epoch 16/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3390 - val_loss: 2.5534
Epoch 17/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3193 - val_loss: 2.5830
Epoch 18/100
52/52 [==============================] - 0s 2ms/step - loss: 2.3137 - val_loss: 2.5539
Epoch 19/100


77/77 [==============================] - 0s 2ms/step - loss: 2.2418 - val_loss: 2.4125
Epoch 44/100
77/77 [==============================] - 0s 3ms/step - loss: 2.2381 - val_loss: 2.4181
Epoch 45/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2358 - val_loss: 2.4188
Epoch 46/100
77/77 [==============================] - 0s 3ms/step - loss: 2.2327 - val_loss: 2.4098
Epoch 47/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2282 - val_loss: 2.4222
Epoch 48/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2262 - val_loss: 2.4122
Epoch 49/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2213 - val_loss: 2.4048
Epoch 50/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2173 - val_loss: 2.4125
Epoch 51/100
77/77 [==============================] - 0s 1ms/step - loss: 2.2148 - val_loss: 2.4096
Epoch 52/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2113 - val_loss: 2.4004
Epoch 53/100


RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000002233C516A30>,
                   param_distributions={'learning_rate': array([0.01236022, 0.00196222, 0.00494489, 0.00453661, 0.00119409,
       0.00743723, 0.00194198, 0.00430075, 0.00142693, 0.02886968,
       0.00387698, 0.00127125, 0.00039058, 0.00799372, 0.00098373,
       0.01051593, 0.0007416 , 0.0237713 , 0.001...
       0.00043384, 0.02907572, 0.01200955, 0.00298007, 0.02456792]),
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85

In [1062]:
print(rnd_search_cv.best_params_)

{'n_neurons': 82, 'n_hidden': 2, 'learning_rate': 0.0006253699615508369}
Model: "sequential_90"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_279 (Dense)           (None, 82)                1148      
                                                                 
 dense_280 (Dense)           (None, 82)                6806      
                                                                 
 dense_281 (Dense)           (None, 1)                 83        
                                                                 
Total params: 8,037
Trainable params: 8,037
Non-trainable params: 0
_________________________________________________________________


## 모델 만들기
- 모델(점수예측)은 회귀모델이기 때문에 출력층에 활성화함수를 지정하지 않았다.

In [1065]:
#모델 만들기

model = Sequential()
model.add(Input(shape=(13,)))
model.add(Dense(400, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(30, activation = 'relu'))
model.add(Dense(1))


## 모델컴파일
- 회귀문제 : MSE(loss) , MAE(eval)
- 이진분류문제 : binary cross entropy(loss) , accuracy(eval)

In [1035]:
#모멘텀(Momentum)은 관성이라는 물리학의 법칙을 응용한 방법입니다. 모멘텀 경사 하강법에 관성을 더 해줍니다. 모멘텀은 경사 하강법에서 계산된 접선의 기울기에 한 시점 전의 접선의 기울기값을 일정한 비율만큼 반영합니다. 이렇게 하면 마치 언덕에서 공이 내려올 때, 중간에 작은 웅덩이에 빠지더라도 관성의 힘으로 넘어서는 효과를 줄 수 있습니다.

In [1066]:
sgd = tf.keras.optimizers.SGD(momentum=0.9)
model.compile(loss='mse',
              optimizer=sgd,metrics=['mae'])


# 텐서보드

In [1067]:

log_dir ='.logs'

tensor_board_cb = [TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)]



# 학습률 스케줄링

In [1068]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

## 모델학습 및 저장

In [1069]:
#최상의 모델을 저장 
cp= ModelCheckpoint('score_predict_best_model.h5', save_best_only=True)
early_Stopping_cb = EarlyStopping(patience=10, monitor='val_loss',restore_best_weights=True)


history = model.fit(x_train,
                    y_train,
                    epochs=500, batch_size=100, validation_data=(x_val,y_val), callbacks=[cp,tensor_board_cb,early_Stopping_cb,lr_scheduler])


Epoch 1/500
25/25 [==============================] - 1s 17ms/step - loss: 18.0102 - mae: 3.1621 - val_loss: 3.8318 - val_mae: 1.5663 - lr: 0.0100
Epoch 2/500
25/25 [==============================] - 0s 14ms/step - loss: 3.8453 - mae: 1.5384 - val_loss: 4.0573 - val_mae: 1.6026 - lr: 0.0089
Epoch 3/500
25/25 [==============================] - 0s 13ms/step - loss: 2.9946 - mae: 1.3359 - val_loss: 2.6294 - val_mae: 1.2583 - lr: 0.0079
Epoch 4/500
25/25 [==============================] - 0s 12ms/step - loss: 2.6999 - mae: 1.2649 - val_loss: 2.4833 - val_mae: 1.2250 - lr: 0.0071
Epoch 5/500
25/25 [==============================] - 0s 12ms/step - loss: 2.5305 - mae: 1.2290 - val_loss: 2.6396 - val_mae: 1.2594 - lr: 0.0063
Epoch 6/500
25/25 [==============================] - 0s 15ms/step - loss: 2.3165 - mae: 1.1820 - val_loss: 2.4479 - val_mae: 1.2041 - lr: 0.0056
Epoch 7/500
25/25 [==============================] - 0s 14ms/step - loss: 2.2475 - mae: 1.1687 - val_loss: 2.4281 - val_mae: 1.19

In [1040]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 30328), started 1:58:45 ago. (Use '!kill 30328' to kill it.)

## 모델 평가

In [1071]:
model.evaluate(X_test,y_test)


25/25 [==============================] - 0s 1ms/step - loss: 2.3280 - mae: 1.1719


[2.327975034713745, 1.171921730041504]

## 예측
- 학습하지 않은 22년 10월 8일 3경기의 데이터를 예측하고자 한다.
- 단 이때 22년 10월 8일의 경기가 진행되지 않음을 가정하고 예측을 진행한다. 스코어예측이나 점수예측은 주로 경기 전에 진행되는
  특성을 따른 것.
- 때문에 모델에 input으로 들어가는 데이터는 각 구단의 직전 3경기의 평균값을 입력하고 실제 22년 10월 8일의 데이터와 비교한다.
- 선발투수는 경기전 확정된 선발투수의 데이터로 입력값을 넣는다(직전 3경기 평균값X 당일 데이터O)
|22-10-8|A구단 점수|B구단 점수|
|------|----------|----------|
|SSG(A) : 삼성(B)|1|6|
|키움 : 두산|5|1|
|한화 : NC|5|6|
|LG : 롯데|2|3|
|KT : KIA|7|2|

In [1042]:
def predict(x):
    tmp = scaler.transform([x])
    score = model.predict([tmp])
    score = round(float(score),2) 
    return score

In [1043]:

SSG = [36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,18,2.49,1.16]
samsung = [41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,19,6.00,1.52]
dusan = [34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,11.6,3.60,1.33]
kiwum =[50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,21.6,2.11,0.95]
hanwha = [42.6,34.6,9,0,5.6,1,6.3,10.6,9,0.258,24.6,4.75,1.42]
NC = [34.3,30.6,6.6,0.3,3,2,8.6,9,7,0.216,11.6,10.64,1.91]
LG = [42.3,38.3,12.6,1,2.3,0.6,8.6,10,7,0.329,16.3,2.31,1.22]
lotte = [35.6,33,8,1,2.3,0,8.6,7,9.3,0.242,16.3,1.86,1.24]
KT = [34.6,33.6,8.3,1.3,0.6,0,9.3,7.3,8.6,0.243,9,5.27,1.74]
KIA = [37.3,34.3,11.3,1.6,2.3,0,4.6,8.3,10,0.328,16.3,2.95,1.22]
score_SSG = predict(SSG)
score_samsung = predict(samsung)
score_dusan = predict(dusan)
score_kiwum = predict(kiwum)
score_hanwha = predict(hanwha)
score_NC = predict(NC)
score_LG = predict(LG)
score_lotte = predict(lotte)
score_KT = predict(KT)
score_KIA = predict(KIA)

print(f"SSG vs 삼성 - {score_SSG} : {score_samsung}")
print(f"키움 vs 두산 - {score_kiwum} : {score_dusan}")
print(f"한화 vs NC - {score_hanwha} : {score_NC}")
print(f"LG vs 롯데 - {score_LG} : {score_lotte}")
print(f"KT vs KIA - {score_KT} : {score_KIA}")

1/1 [==============================] - 0s 20ms/step


C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step


C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 28ms/step
SSG vs 삼성 - 1.73 : 5.89
키움 vs 두산 - 6.17 : 3.6
한화 vs NC - 4.05 : 4.37
LG vs 롯데 - 6.66 : 1.94
KT vs KIA - 3.52 : 4.89


C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [1044]:
SSG = [30,30,3,1,0,0,6,8,13,0.1,5,2.49,1.16]
samsung = [35,31,10,2,1,3,5,8,8,0.322,11,6.00,1.52]
dusan = [30,30,5,1,0,0,9,8,8,0.166,6,3.60,1.33]
kiwum =[39,32,10,1,3,1,2,5,15,0.312,12,2.11,0.95]
hanwha = [52,44,9,1,7,1,11,9,15,0.204,22,4.75,1.42]
NC = [48,45,12,1,3,0,11,11,11,0.266,17,10.64,1.91]
LG = [39,35,9,0,4,0,8,10,8,0.257,26,2.31,1.22]
lotte = [38,31,9,1,5,1,5,11,6,0.29,19,1.86,1.24]
KT = [40,36,9,2,3,1,7,1,19,0.25,15,5.27,1.74]
KIA = [39,34,8,0,4,1,10,7,9,0.235,22,2.95,1.22]
score_SSG = predict(SSG)
score_samsung = predict(samsung)
score_dusan = predict(dusan)
score_kiwum = predict(kiwum)
score_hanwha = predict(hanwha)
score_NC = predict(NC)
score_LG = predict(LG)
score_lotte = predict(lotte)
score_KT = predict(KT)
score_KIA = predict(KIA)

print(f"SSG vs 삼성 - {score_SSG} : {score_samsung}")
print(f"키움 vs 두산 - {score_kiwum} : {score_dusan}")
print(f"한화 vs NC - {score_hanwha} : {score_NC}")
print(f"LG vs 롯데 - {score_LG} : {score_lotte}")
print(f"KT vs KIA - {score_KT} : {score_KIA}")

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 23ms/step


C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 35ms/step
SSG vs 삼성 - 0.69 : 6.53
키움 vs 두산 - 6.4 : 1.43
한화 vs NC - 5.46 : 6.01
LG vs 롯데 - 1.85 : 4.08
KT vs KIA - 5.04 : 1.86


C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
